In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Data loading

Not every feature seems to be meaningful -> train dataset should be investigated for redundant features. </br>
I'm going to use two approaches to find useful features:
1. **Chi-Square Test** to exclude correlated categorical features
1. **Exploring LogReg coefficients** to find out useful and useless features


In [ ]:
column_dtypes = {'IsBeta' : 'Int8',
                 'RtpStateBitfield' : 'Int16',
                 'AVProductsInstalled' : 'Int8',
                 'AVProductsEnabled' : 'Int8',
                 'IsSxsPassiveMode' : 'Int8',
                 'HasTpm' : 'Int8',
                 'CountryIdentifier' : 'Int64',
                 'CityIdentifier' : 'Int64',
                 'IsProtected' : 'Int8',
                 'AutoSampleOptIn' : 'Int8',
                 'SMode' : 'Int8',
                 'OsSuite' : 'Int16',
                 'Firewall' : 'Int8',
                 'Census_ProcessorManufacturerIdentifier' : 'Int8',
                 'Census_HasOpticalDiskDrive' : 'Int8',
                 'Census_IsPortableOperatingSystem' : 'Int8',
                 'Census_IsFlightsDisabled' : 'Int8',
                 'Census_IsSecureBootEnabled' : 'Int8',
                 'Census_IsWIMBootEnabled' : 'Int8',
                 'Census_IsVirtualDevice' : 'Int8',
                 'Census_IsTouchEnabled' : 'Int8',
                 'Census_IsPenCapable' : 'Int8',
                 'Census_IsAlwaysOnAlwaysConnectedCapable': 'Int8',
                 'Census_OSInstallLanguageIdentifier' : 'Int8',
                 'Census_OSUILocaleIdentifier' : 'Int16',
                 'Wdft_IsGamer' : 'Int8',
                 'HasDetections' : 'int'}

In [ ]:
columns_to_ignore = ('DefaultBrowsersIdentifier',  # 95.14% NA values
                     'PuaMode', # 99.97% NA values
                     'Census_ProcessorClass', # 99.59% NA values.
                     'Census_InternalBatteryType', # 71.05% NA values
                     'Census_IsFlightingInternal', #83.04% NA values
                     'Census_ThresholdOptIn', # 63.52% NA values
                     
                     # numerical features
                     'Census_PrimaryDiskTotalCapacity', 
                     'Census_SystemVolumeTotalCapacity', 
                     'Census_TotalPhysicalRAM',        
                     'Census_InternalPrimaryDisplayResolutionHorizontal',
                     'Census_InternalPrimaryDisplayResolutionVertical',
                     'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                     'Census_InternalBatteryNumberOfCharges'
                    )

In [ ]:
label_col = 'HasDetections'

In [ ]:
train_df = pd.read_csv('/kaggle/input/microsoft-malware-prediction/train.csv', 
                       usecols = lambda x: x not in columns_to_ignore,
                       dtype = column_dtypes)
train_df.head()

## 2. Data investigation and preprocessing 

### 2.1 Preprocessing

In [ ]:
print(f'Train DataFrame Shape : {train_df.shape}')

In [ ]:
# (1 / 8921483) * 100% = 1.12e-5%
# so we need 5 digit after a decimal point to find out at least one missing value

for col in train_df.columns:
    print(f'"{col}" has {train_df[col].nunique()} unique values and {train_df[col].isna().sum() / train_df.shape[0] * 100:.5f}% NA values.')

In [ ]:
sns.countplot(x = train_df[label_col])

In [ ]:
train_df = train_df.drop(columns = ['MachineIdentifier'])

In [ ]:
import sys

def preprocess_cat_values(df):
    temp = df.copy()
    
    cols = temp.select_dtypes(include = [object]).columns.tolist()   
    
    # use 'sys.intern' to reduce memory use
    # see why here: https://stackoverflow.com/questions/76104472/python-str-lower-causes-memory-leak
    temp[cols] = temp[cols].astype(str).apply(lambda x: x.str.lower().apply(sys.intern))
    
    os_build_lab_cat = 'OsBuildLab'
    if os_build_lab_cat in temp.columns:
        os_build_lab_df = temp[os_build_lab_cat].str.split(pat = '.', n = 5, expand = True)
        os_build_lab_df = os_build_lab_df.astype(str).apply(lambda x: x.str.lower().apply(sys.intern))
        os_build_lab_df = os_build_lab_df.add_prefix(os_build_lab_cat + '_')
        
        temp = pd.concat([temp, os_build_lab_df], axis = 1)
        temp = temp.drop(columns = os_build_lab_cat)
    
    smart_screen_cat = 'SmartScreen'
    if smart_screen_cat in temp.columns:
        temp.loc[temp[smart_screen_cat] == 'promt', smart_screen_cat] = 'prompt'
        temp.loc[temp[smart_screen_cat] == '00000000', smart_screen_cat] = '0'
        temp[smart_screen_cat] = temp[smart_screen_cat].astype(str).apply(sys.intern)
        
    disk_type_cat = 'Census_PrimaryDiskTypeName'
    if disk_type_cat in temp.columns:
        disk_types = ['HDD', 'SSD']
        temp.loc[~temp[disk_type_cat].isin(disk_types), disk_type_cat] == 'na'                                
        temp[disk_type_cat] = temp[disk_type_cat].astype(str).apply(sys.intern)
        
    role_name_cat = 'Census_PowerPlatformRoleName'
    if role_name_cat in temp.columns:
        na_types = ['unspecified', 'unknown', np.nan]
        temp.loc[temp[role_name_cat].isin(na_types), role_name_cat] == 'na'                             
        temp[role_name_cat] = temp[role_name_cat].astype(str).apply(sys.intern)
    
    return temp

In [ ]:
train_df = preprocess_cat_values(train_df)
train_df.shape

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df = train_df.drop_duplicates()
train_df.shape

### 2.2 Take a closer look at some of the features

In [ ]:
train_df_without_na = train_df.dropna()

In [ ]:
bin_features = [col for col in train_df_without_na.columns if train_df_without_na.loc[:, col].nunique() <= 2]
bin_features.remove(label_col)
bin_features

In [ ]:
plt.figure(figsize=(10, 4 * len(bin_features)))
for i, col in enumerate(bin_features, start = 1):
    plt.subplot(len(bin_features), 1, i)
    sns.countplot(x = train_df_without_na[col], hue = train_df_without_na[label_col])

In [ ]:
bin_features_to_remove = ['IsBeta',
                          'IsSxsPassiveMode',
                          'HasTpm',
                          'AutoSampleOptIn',
                          'SMode',
                          'Firewall',
                          'Census_DeviceFamily',
                          'Census_IsPortableOperatingSystem',
                          'Census_IsFlightsDisabled',
                          'Census_IsWIMBootEnabled',
                          'Census_IsVirtualDevice'
                         ]

In [ ]:
multi_cat_features = [col for col in train_df.columns if col not in bin_features]
multi_cat_features.remove(label_col)
multi_cat_features

In [ ]:
plt.figure(figsize=(10, 4 * len(multi_cat_features)))
for i, col in enumerate(multi_cat_features, start = 1):
    plt.subplot(len(multi_cat_features), 1, i)
    sns.countplot(x = train_df_without_na[col], hue = train_df_without_na[label_col])

In [ ]:
multi_cat_features_to_remove = ['ProductName',
                                'RtpStateBitfield',
                                'AVProductStatesIdentifier',
                                'AVProductsEnabled',
                                'UacLuaenable']

### 2.3 Chi-Square Test 

I'm going to use Chi-Square Test to assess independence among categoriacal features where: </br>
**Ho** — feature1 and feature2 are independent</br>
**Ha** — There is relationship between feature1 and feature2</br></br>

So if **p-value < alpha (I'll take 0.05 as alpha)** - Ho can be rejected and I assume that there is some relationship between columns.</br>
But if **p-value > 0.05** - Ho cannot be rejected so I can assume that investigated columns are independent.

In [ ]:
train_df_without_na = train_df_without_na.drop(columns = bin_features_to_remove + multi_cat_features_to_remove)
train_df_without_na = train_df_without_na.sample(n = 100_000)

In [ ]:
from itertools import combinations

In [ ]:
import scipy.stats as ss

In [ ]:
def get_p_value(x, y):
    return ss.chi2_contingency(pd.crosstab(x, y))[1]

In [ ]:
alpha = 0.05

In [ ]:
col_pairs = []
for col1, col2 in combinations(train_df_without_na.columns, 2):
    p_value = get_p_value(train_df_without_na[col1], train_df_without_na[col2])
    if p_value <= alpha:
        col_pairs.append((col1, col2))

In [ ]:
from collections import Counter

cols_to_check = [col for pair in col_pairs for col in pair]
cols_counter = Counter(cols_to_check)
cols_counter.most_common()

In [ ]:
# apparently Chi-Square test doesn't suit here
# because after removing all 'dependent' features and then all left duplicated rows
# we'll get a really small dataframe with insufficient information to fit a good model

len(cols_counter), len(train_df.columns)

In [ ]:
del train_df_without_na  

Lets investigate correlation only between columns and target:

In [ ]:
#train_df = train_df.drop(columns = bin_features_to_remove + multi_cat_features_to_remove)

In [ ]:
independent_cols = []

for col in train_df.columns:
    if col == label_col: continue
        
    p_value = get_p_value(train_df[col], train_df[label_col])
    
    # collect independent cols
    if p_value > alpha:
        independent_cols.append(col)
        
independent_cols

## 3. OHE

In [ ]:
train_df = train_df.drop(columns = independent_cols)

In [ ]:
y = train_df.pop(label_col)
y.shape

In [ ]:
# OHE
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', drop = 'first')
X_train_ohe = ohe.fit_transform(train_df.astype('category'))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train_ohe, y, test_size = 0.2, random_state = 42)

In [ ]:
del X_train_ohe, train_df 

## 4. LogReg

In [ ]:
# LogReg

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 7000, random_state = 42)
lr.fit(X_train, y_train) 

In [ ]:
lr.score(X_val, y_val)

In [ ]:
coef_df = pd.DataFrame({'name' : list(ohe.get_feature_names_out()), 
                        'coef': lr.coef_[0]}).sort_values(by = 'coef', key = lambda x : abs(x))
coef_df

In [ ]:
mask = coef_df['coef'] < 1e-9 

col_names = ('Census_OSBranch', 'Census_OSSkuName', 'OsBuildLab')
def correct_feature_name(name):
    for col in col_names:
        if name.startswith(col):
            if col == 'OsBuildLab':
                return '_'.join(name.split('_')[:2])
            return col
    return name


bad_col_list = [correct_feature_name(feature_name[0]) for feature_name in coef_df.loc[mask, 'name'].str.rsplit(pat = '_', n = 1).values]
good_col_list = [correct_feature_name(feature_name[0]) for feature_name in coef_df.loc[~mask, 'name'].str.rsplit(pat = '_', n = 1).values]

In [ ]:
bad_counter = Counter(bad_col_list)
bad_counter.most_common()

In [ ]:
good_counter = Counter(good_col_list)
good_counter.most_common()

In [ ]:
set(bad_counter.keys()) & set((good_counter.keys()))

In [ ]:
set(bad_counter.keys()) - set((good_counter.keys()))

In [ ]:
del X_train, X_val, y_train, y_val

## 5. Test

In [ ]:
test_df = pd.read_csv('/kaggle/input/microsoft-malware-prediction/test.csv',
                      usecols = lambda x: (x not in columns_to_ignore) and (x not in independent_cols),
                      dtype = column_dtypes)
test_df.head()

In [ ]:
sub = test_df.pop('MachineIdentifier').to_frame()

In [ ]:
test_df = preprocess_cat_values(test_df)

In [ ]:
X_test_ohe = ohe.transform(test_df.astype('category'))

In [ ]:
del test_df

In [ ]:
# predictions
sub[label_col] = lr.predict_proba(X_test_ohe)[:, 1]
sub.head()

In [ ]:
sub.to_csv('submission.csv', index = False)